# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,7,1,484,True
1,0,7,2,492,True
2,0,7,3,476,True
3,0,7,4,425,True
4,0,7,5,478,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,491.5,416.888889
1,3,0,False,466.5,414.666667
2,4,0,False,469.0,421.666667
3,5,0,False,481.5,400.666667
4,6,3,True,514.0,532.555556
...,...,...,...,...,...
5579,9990,0,False,484.0,423.666667
5580,9995,0,False,487.5,403.000000
5581,9997,0,False,494.0,416.333333
5582,9998,0,False,480.5,412.333333


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,491.5,416.888889,35,1,1
1,3,0,0,466.5,414.666667,20,1,2
2,4,0,0,469.0,421.666667,26,0,1
3,5,0,0,481.5,400.666667,29,0,2
4,6,3,1,514.0,532.555556,66,0,2


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

[04.04.2023 10:52:29 | matcher | INFO]: Applying matching
[04.04.2023 10:52:29 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [ 0. 43.  0.  5.  0. 17.  7.  0.  3.  9.  7. 30.  0. 20. 14. 11. 36.  0.
  2.  1.  2.  4.  0.  0.  8.  2.  0.  0.  4. 15.  1.  0. 18. 11.  6.  0.
  1.  0.  2. 29.  9.  2.  6. 32.  0.  0.  3. 12.  0. 19.  0.  8.  0. 11.
  0.  5. 27.  5. 14.  6.  2. 47.  4.  0.  0.  3.  6. 10.  7.  0. 42.  1.
  0. 13.  3. 41. 15.  7.  0. 11.  0. 14.  2. 28.  0. 36.  8.  0. 11.  0.
 18.  6.  0. 13. 17. 12. 19.  0.  2.  3.  0. 13. 36. 20. 15. 44.  0. 13.
 12.  6.  9.  0. 21.  5.  7.  0.  0.  0.  9.  8. 11.  0. 12.  0.  0. 26.
  0.  0.  9. 20.  3. 48. 14.  1.  4.  9. 13. 11.  1.  3.  3.  3. 15.  2.
 79. 27. 16.  0.  1.  0. 22. 12.  0. 12.  1. 12. 15. 12. 17.  0.  0.  0.
  4. 18. 35. 24.  6.  0.  0.  0.  5.  0.  7.  3.  5. 12.  8. 68.  7.  1.
  4.  2.  9. 30. 99. 12. 28. 20.  0. 10. 26.  4.  4. 17. 26.  8.  2.  5.
  2. 20. 16.  0. 17.  0.

Finding index
Finding index


[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.0
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.024707
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.001247
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.006837
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.000181
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.00195
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.004043
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.00074
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.00096
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.000388
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.006783
[04.04.2023 10:52:30 | psi_pandas | INFO]: PSI values: 0.05
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 9.209419
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.688332
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi 

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

[04.04.2023 10:52:30 | matcher | INFO]: Applying matching
[04.04.2023 10:52:30 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [ 9. 28.  2.  0. 12.  0.  0. 17.  6.  9.  0. 17.  1. 11.  0.  0.  0.  4.
 20.  2.  2.  0. 15.  0. 28. 10.  7. 14.  0. 57. 40.  1. 46. 43.  0.  6.
 22.  0.  0.  0.  5. 10. 10.  2. 22.  6.  0.  0.  0.  0.  3.  6.  0. 21.
  0.  0. 19. 19.  7. 16.  5. 16.  9. 10.  7.  6.  1.  0.  0.  9.  0. 14.
 38. 26.  2.  0. 27.  1.  0.  5.  0. 13.  0. 16.  0. 63. 15. 14.  0.  0.
 21.  7.  0. 29.  0.  5. 20.  0.  0.  4. 42. 14. 64. 61. 16.  4. 13.  0.
  3. 13.  0. 41.  0. 14.  0.  0.  6.  0.  4. 33. 44.  0.  4. 13. 15. 31.
 14.  0.  5.  0. 18.  0.  1. 32.  7.  0.  8. 48. 30. 12. 22. 12.  0. 28.
 45.  5. 10.  3.  5.  0. 15.  9.  0. 26. 28.  0.  7.  5. 10.  5. 13. 26.
 43.  4. 27.  9.  3.  1.  6. 20.  1.  7. 29. 13. 31. 29.  4.  5.  8. 13.
  9. 27. 24. 22. 18.  2. 32. 43.  1. 21.  6. 16.  7.  2.  1.  0. 21.  0.
 20.  4.  0. 53.  4. 34.

Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:52:30 | psi_pandas | INFO]: PSI values: 0.08
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 9.209419
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.688332
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.635979
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.723482
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.688332
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.688332
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.705883
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.688332
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.67083
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.705883
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.705883
[04.04.2023 10:52:30 | psi_pandas | INFO]: PSI values: 16.11
[04.04.2023 10:52:30 | psi_pandas | INFO]: Creating report
[04.04.2023 10:52:30 | psi_pandas | INFO]: sub_psi value is 0.

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

[04.04.2023 10:52:30 | matcher | INFO]: Counting feature importance
[04.04.2023 10:52:30 | lama_feature_selector | INFO]: Getting feature scores


,Feature,Importance
0,pre_spends,704997.921875
1,age,594104.640137
2,is_male,29601.600586


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

[04.04.2023 10:52:35 | matcher | INFO]: Applying matching
[04.04.2023 10:52:35 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [  0.   1.   0.  25.   5.   4.  11.  14.  17.  19.   0.  13.   2.   0.
  10.   4.  38.  25.  18.  16.   4.  13.   9.   3.   8.   6.   8.   5.
  25.   0.   6.   2.  21.  12.   1.   1.  16.   4.   6.  24.   0.   5.
   2.   1.  15.   8.  18.   2.   0.   9.  15.  22.   0.   9.  20.  18.
   9.  32.   2.  15.   4.  15.  28.   4.   1.   1.   1.   8.   0.   4.
  22.   7.  15.  23.   2.  13.   6.   3.  10.   2.  16.   5.   0.  39.
   0.  15.   5.   8.  17.  24.  24.   2.   3.  15.   0.   0.  20.   2.
   3.   3.  46.   8.  42.  17.  43.   1.  17.   5.   2.   5.   7.  10.
  10.  21.  36.   7.   3.   0.   1.  24.  10.   5.  11.  13.  39.  22.
   7.   0.  19.  22.   3.   5.  12.  15.   4.  16.  14.   0.  27.  10.
   8.  25.   1.  31.  25.  18.  25.   1.   2.  20.   0.   0.   5.  19.
  45.   8.  32.   0.   8.   8.   5.  27.  28. 

Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.0
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.000789
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.00523
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.139271
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.029084
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.018758
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.042632
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.003496
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.008252
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 5.3e-05
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.012155
[04.04.2023 10:52:35 | psi_pandas | INFO]: PSI values: 0.26
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.0
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi value is 0.013582
[04.04.2023 10:52:35 | psi_pandas | INFO]: sub_psi valu

In [14]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,3090,5498,3,507.0,41,1,1,460,820,0,490.0,47,1,1,416.888889,535.222222,535.488116,0,1
1,3050,5426,3,471.0,64,1,1,2721,4848,0,468.0,22,1,1,420.000000,509.222222,509.192781,0,1
2,3035,5402,3,501.5,63,0,1,2696,4795,0,509.5,30,1,1,431.333333,516.777778,516.093356,0,1
3,3045,5416,3,472.5,62,0,1,3076,5471,0,488.5,28,0,1,428.222222,509.000000,509.131432,0,1
4,3172,5647,3,467.5,55,1,1,1361,2401,0,464.0,30,1,1,411.888889,514.333333,515.040399,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,1108,1976,0,496.0,54,0,2,2115,3773,3,497.5,59,0,2,521.555556,416.111111,416.089938,1,0
5580,1106,1970,0,500.5,47,1,2,679,1228,3,467.0,52,0,2,517.000000,421.000000,421.201960,1,0
5581,3322,5944,0,499.5,63,1,2,307,531,3,462.0,64,0,2,508.555556,426.222222,426.081883,1,0
5582,3324,5949,0,470.5,23,0,2,2256,4025,3,501.5,23,1,2,515.333333,417.888889,417.744075,1,0


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,99.252111,1.048398,0.0,97.197252,101.306970
ATC,99.216595,1.113312,0.0,97.034504,101.398685
ATT,99.656336,0.926445,0.0,97.840504,101.472167


### 3.2 SMD, PSI, KS-test, repeats

In [17]:
# matching quality result - SMD
model.quality_result

,value
match_control_to_treat,0.71
match_treat_to_control,0.05


In [ ]:
# matching quality result - PSI
model.quality_result['psi']

In [ ]:
# matching quality result - KS-test

model.quality_result['ks_test']

In [ ]:
# matching quality result - repeats
model.quality_result['repeats']

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [18]:
 
model.validate_result()
 

[04.04.2023 10:56:09 | matcher | INFO]: Applying validation of result
[04.04.2023 10:56:09 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [ 4. 17. 13. 12. 16. 11.  4.  8.  4.  3. 24.  4.  0. 16.  8. 14.  8. 11.
  4. 12.  6.  3.  4.  3. 10.  4.  7. 24. 18. 16.  3.  7.  0.  9. 13. 13.
 13. 43. 12. 11. 13. 33. 10.  7. 13. 15.  7. 32. 22. 11. 24. 14.  8.  7.
  6. 24.  8. 20.  6. 11.  8. 23. 23.  0.  9. 23. 28.  3.  4. 26. 19. 29.
  8. 12. 13. 25. 13.  8.  8. 22.  7. 25.  7.  3.  1. 30.  8.  4. 15.  8.
 37. 29.  8. 20.  6.  2. 10.  0. 33. 10.  2.  0.  2.  7.  2. 23. 11. 19.
 16.  7. 15. 14. 16.  5.  1. 10. 13. 25. 22. 12.  7. 18.  7. 17. 13. 21.
  3. 12.  8.  5.  9. 15.  6.  8. 26.  4.  4. 11.  5.  3. 14. 10.  3. 10.
  8.  9.  8.  2.  7. 14. 13.  6.  5.  4. 34. 23.  4. 31. 10. 12. 19. 38.
  5.  0.  6. 23. 16.  5.  8.  2.  4.  3.  9. 15. 17. 23. 11. 17.  3. 26.
 13.  8. 19.  6. 33.  5.  4. 14. 25.  2. 13. 17. 16.  8.  7. 18. 13. 14.
 14.  6.  8.

Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:56:09 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [  0.  11.  10.  27.   3.   4.   6.  22.  14.   6.   0.  18.   3.  13.
  20.   6.  25.  33.  22.  18.  44.  10.   5.  15.   1.  15.   9.  21.
   7.   4.  10.  10.  10.  64.  24.  14.  22.  18.   4.   3.   1.   3.
   5.  23.  11.  17.   0.  21.   0.  18.   2.  14.   8.   7.  32.   6.
  22.  23.  12.  11.  24.   2.   7.  10.  32.  14.  22.  10.  36.  20.
  23.  16.   8.   7.   5.   8.  11.  11.   2.  13.   2.  10.   0.   0.
   8.   2.   0.   0.   5.  10.  15.   8.   1.  22.   6.   3.  38.   0.
   6.  14.  20.   6.  13.  28.   7.  67.   6.  11.  18.   2.   0.   7.
   6.   8.  30.   0.   9.  10.  13.   2.   8.  18.   5.   5.   3.   1.
   9.  28.   2.  14.   6.  35.   4.   4.  60.   8.   2.  12.  33.  22.
   3.  27.   5.  10.   5.   7.   6.   2.   9.   7.   9.  11.  39.   2.
  19.   0.   7.  12.   5.   4.  31.  10.   5.  33.  16.  10.   6.  23.
  20.  10.   1.   0.   0.  41.   

Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:56:10 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [  9.  26.  25.  32.  24.  22.  15.   6.   8.  11.  11.  34.  15.   0.
   5.   9.  10.   6.   6.  14.  13.   4.  11.   2.  10.  23.   2.  23.
   7.   0.   2.  21.   8.  10.   8.   6.   4.  35.   0.  39.   6.   9.
  10.  17.  29.   9.   0.  10.  24.  34.   5.   5.   3.   7.   0.  15.
   7.  24.  17.   8.  14.  16.   4.   9.   6.  10.   9.   2.  17.   0.
  43.  12.  37.  30.  10.  16.   6.  14.  11.  21.   8.  55.   7.   4.
   7.  26.  11.  23.   7.   6.  11.  10.   7.  18.  42.  28.  13.  25.
  16.  10.  23.  17.  35.  11.  10.  13.  17.  19.   0.   3.   0.  12.
   2.   0.   7.   1.   1.   3.   5.  11.   3.  19.   7.  15.   0.  12.
  10.  21.   5.   0.   4.  19.  12.  16.   9.  15.  11.   2.   9.  13.
  27.   9.   6.   6.  16.  39.   7.   7.   3.  29.  21.   2.   6.  16.
   8.   7.   0.   6.  10.   4.  11.  12.  21.   5.  10.  19.   3.  10.
   0.  16.  30.   7.  20.  10.   

...

Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:56:10 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [ 7.  5.  9. 12. 15. 10. 34.  3.  7. 24.  5. 34. 12.  2.  0. 17.  0. 48.
 31. 38. 33.  0.  8.  0.  5. 26.  7.  9.  3. 19.  0. 10. 17. 18.  9. 16.
 15. 14.  0.  2.  4.  0.  0. 16. 28. 28.  9.  7. 11. 27.  3. 12. 34. 18.
  0.  7. 35.  9.  5. 13. 16.  0.  0. 12.  8. 18.  1.  0. 31.  2.  6.  3.
  9. 18. 17. 14.  4. 38.  9. 38. 48. 23.  8. 21. 24.  9. 23.  6.  7. 35.
 30.  9. 10. 13. 62. 10.  0. 19.  7. 12. 23.  0.  3. 15.  0.  1.  4. 15.
 14.  3.  3.  7.  6.  4.  5.  3. 17. 10.  3. 43.  7.  4. 34.  0. 15.  4.
  6.  5.  4.  1. 37.  3. 18. 20.  0. 61.  1. 19. 11.  0. 39.  0.  0.  2.
 55.  4. 19.  6.  1.  0.  0. 24.  2. 65. 13.  5.  4. 25.  3.  1.  8. 31.
 28. 19.  6. 14. 26.  1.  2. 22. 23.  3.  4. 14. 42. 16.  8.  0.  5.  8.
  0. 10.  8. 19.  0.  8.  0.  1. 29.  1.  2. 18.  4.  0.  1. 42. 40.  1.
 17.  9.  3.  5. 18. 11. 14. 10.  6. 55. 10. 26. 23.  5.  4.  2.  0. 20.
 34. 28. 

Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index


[04.04.2023 10:56:10 | Faiss matcher | INFO]: Calculated the number of times each subject has appeared as a match: [  0.   0.  13.  83.  75.   0.  91.  10.   0.   0.  74.   0.   0.   0.
   0.   0.   1.   0.  29.   0.  66.   0.  13.   0.   1.  82.   4.   0.
   0.  47.   0.   0.   0.  81.   0.   0.   0.   0.   0.   1.   0.   0.
   0.  25.  10.   3.   8.   0.  74.  15.  78.   0.   0.   5.   0.   1.
   6.   0.  10.   7.   0.   0.   0.   0.   0.   0.   0.  39.   4.   0.
  20.  37.  19.  34.   0.  10.   0.   0.   0.  96.  24.   0.   0.  51.
  11.   0.   0.   2.  11.   0.   0.   1.   0.   0.  67.  72.   0.  12.
  45.   1.   0.   0.   0.   7.   8.   0.  68.   0.   0.   0.   0.   1.
   0.   0.   1.   0.  29.   0.   0.   0.   0.   0.   0.   8.  50.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  27.
  27.   0.   1.   4.  65.   0.  13.   0.  12.  36.   0.  22.   0.   0.
  22.   0.  13.   6.   9.   0.  24.   6.   0.   0.   1.   6.   0.   0.
   0.  21.   0.   0.  42.   0.  9

{'post_spends': 1.3168330093649983}

In [19]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,3096,5505,0,479.0,18,1,1,2945,5243,0,480.0,24,1,1,416.888889,419.555556,419.568179,0,1
1,3035,5402,3,501.5,63,0,1,2730,4864,0,492.5,53,0,1,420.000000,401.888889,402.165504,0,1
2,3159,5620,0,482.5,59,0,1,3108,5521,0,495.5,60,0,1,431.333333,429.333333,428.993187,0,1
3,3123,5550,0,504.5,46,0,1,2241,4005,0,497.0,37,0,1,428.222222,412.000000,416.612467,0,1
4,2780,4946,3,490.5,60,0,1,294,510,3,494.0,56,0,1,411.888889,408.777778,413.700716,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,3317,5930,3,476.0,45,1,2,3782,6766,0,465.5,54,1,2,425.777778,423.888889,422.860657,1,0
5580,1108,1976,0,496.0,54,0,2,1373,2419,0,487.0,26,0,2,428.888889,402.777778,403.957425,1,0
5581,3322,5944,0,499.5,63,1,2,2281,4070,0,465.0,46,0,2,415.555556,424.111111,422.181634,1,0
5582,3324,5949,0,470.5,23,0,2,1894,3356,0,504.5,42,1,2,421.777778,519.111111,520.629042,1,0
